# KBO 타자정보 크롤러
https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx

In [1]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import random
import pandas as pd
import os

## KBO 타자 기록실 연결

In [2]:
def connect_KBO():
    global driver
    global act
    
    # 드라이버 연결
    driver = webdriver.Chrome()
    
    # 액션체인 객체 생성
    act = ActionChains(driver)
    
    # KBO 기록실 이동 - 타자 / 투수
    driver.get(f'https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
    
    # 대기
    driver.implicitly_wait(3)

## 연도별 선수 URL 수집기

In [3]:
def get_player_urls(year):
    player_urls = list()
    
    # 연도 정보 변경
    year_btn = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlSeason_ddlSeason"]')
    sel_year_btn = Select(year_btn)
    sel_year_btn.select_by_value(str(year))
    driver.implicitly_wait(random.randrange(3, 7))
    
    
    # 활성화된 탭만큼 이동
    tabs = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/div')
    
    for i in tabs.find_elements(By.TAG_NAME, "a")[1:-1]:
        i.click()
        driver.implicitly_wait(random.randrange(3, 7))
        table = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[3]/table/tbody')
        player_list = table.find_elements(By.TAG_NAME, "a")

        for j in player_list:
            player_urls.append(j.get_attribute("href"))

    return player_urls

## 선수 정보 취합기

In [4]:
def get_players_info(urls):
    players_info = list()

    for i in urls:
        driver.implicitly_wait(random.randrange(2,5))
        driver.get(i)

        player_info = driver.find_elements(By.XPATH, '//*[@id="contents"]/div[2]/div[1]/div[1]/ul')
        
        # 은퇴선수 정보 pass
        if "은퇴선수" in driver.title:
            continue
        
        player_team = driver.find_element(By.XPATH, '//*[@id="h4Team"]').text.split()[0]

        player_name = player_info[0].find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblName")[0].text # 선수이름
        player_birth = player_info[0].find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblBirthday")[0].text.replace("년 ", "-").replace("월 ", "-").replace("일", "") # 선수 생년월일
        player_height, player_weight = player_info[0].find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblHeightWeight")[0].text.split("/") # 선수 신장/체중
        player_position = player_info[0].find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblPosition")[0].text # 선수 포지션
        player_salary = player_info[0].find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblSalary")[0].text # 선수 연봉
        
        # hitter_name, hitter_team, hitter_birth, hitter_height, hitter_weight, hitter_position, hitter_salary
        players_info.append([player_name, player_team, player_birth, player_height, player_weight, player_position, player_salary])
        driver.implicitly_wait(random.randrange(1,3))

    return players_info

## 크롤러 실행기 - URL 패치 필요

In [8]:
def runner(year):
    # 필요한 데이터 피쳐 리스트
    hitter_col = ["hitter_name", "hitter_team", "hitter_birth", "hitter_height", "hitter_weight", "hitter_position", "hitter_salary"]
    f_name = f"타자정보_{year}.csv"
    print(f"현재 경로: [{os.getcwd()}]에 [{f_name}] 로 저장예정. \n원치 않으면 실행을 중단하세요.")
        
    connect_KBO()
    #url 
    urls = get_player_urls(year)
    print(urls)
    #data = get_players_info(urls)
    
    #df = pd.DataFrame(data, columns=hitter_col)
    #df.to_csv(f_name, index=False)
    
    print(f"파일 저장 완료\n파일 저장 경로: {os.getcwd()}/{f_name}")
    
    return df

In [10]:
runner(2021) # 크롤링 희망 연도

현재 경로: [/Users/dokw/Library/CloudStorage/GoogleDrive-playdev7@gmail.com/내 드라이브/GJAI/StudyNote/호크아이]에 [타자정보_2021.csv] 로 저장예정. 
원치 않으면 실행을 중단하세요.
['https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=77532', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=53827', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=66108', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=76232', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=67304', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=67341', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=75847', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=69995', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=68119', 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerI

NameError: name 'df' is not defined